In [ ]:
import json
import os
import re
import math
from datetime import datetime, timedelta
from collections import defaultdict
from decimal import Decimal

import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.io import to_html

import yfinance as yf

from IPython.display import display, HTML

from transaction_history_processor import PortfolioHistory

In [ ]:
if not os.path.exists('portfolio_history.json'):
    print("portfolio_history.json not found. Generating it now...")
    portfolio_history = PortfolioHistory()
    portfolio_history.process_transaction_history(
        input_file='transaction_history.json',
        save_output=True,
        output_file='portfolio_history.json'
    )
    print("portfolio_history.json has been generated.")

with open('portfolio_history.json', 'r') as f:
    data = json.load(f)
    sectors = data['sectors']
    portfolio_data = data['portfolios']

In [ ]:
def plot_stock_proportions(portfolio_data, date=None, show_values=True, main_threshold=3.0):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return None

    holdings = portfolio_data[date]['holdings']
    labels = list(holdings.keys())
    values = [float(data['market_value']) for data in holdings.values()]

    total = sum(values)
    percentages = [(value / total) * 100 for value in values]

    main_holdings = list(zip(labels, values, percentages))
    main_holdings.sort(key=lambda x: x[2], reverse=True)

    other_holdings = [h for h in main_holdings if h[2] < main_threshold]
    if len(other_holdings) > 1:
        others_value = sum(v for _, v, _ in other_holdings)
        others_percentage = sum(p for _, _, p in other_holdings)
        other_symbols_info = "<br>".join(f"<span style='font-family: monospace;'>{l:<5}\t{
                                         p:>6.2f}%</span>" for l, _, p in other_holdings)
        main_holdings = [h for h in main_holdings if h[2] >=
                         main_threshold] + [("Others", others_value, others_percentage)]

    main_labels, main_values, main_percentages = zip(*main_holdings)

    # Create custom hover text
    hover_text = [f"{label}<br>${value:,.2f}<br>{percentage:.2f}%" for label,
                  value, percentage in zip(main_labels, main_values, main_percentages)]
    if "Others" in main_labels:
        others_index = main_labels.index("Others")
        hover_text[others_index] += f"<br><br>{other_symbols_info}"

    fig = go.Figure(data=[go.Pie(labels=main_labels,
                    values=main_percentages,
                    textinfo='label+percent',
                    hovertext=hover_text,
                    hoverinfo='text')])

    fig.update_layout(
        title=f'Stock Proportions by Value on {date}',
        showlegend=True,
        legend=dict(title="Holdings")
    )

    if show_values:
        fig.update_traces(
            text=[f'${value:,.2f}' for value in main_values], textposition='inside')

    return fig

# Call the function
plot_stock_proportions(portfolio_data, show_values=False)

In [ ]:
# plot_stock_proportions(
#     portfolio_history, date="2023-10-26")

In [ ]:
def plot_sector_proportions(portfolio_data, sectors, date=None):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return None

    holdings = portfolio_data[date]['holdings']
    sector_counts = {}
    for symbol, data in holdings.items():
        sector = sectors.get(symbol, 'ETF')
        if sector not in sector_counts:
            sector_counts[sector] = 0
        sector_counts[sector] += float(data['quantity']) * \
            float(data['market_price'])

    labels = list(sector_counts.keys())
    sizes = list(sector_counts.values())

    fig = go.Figure(
        data=[go.Pie(labels=labels, values=sizes, textinfo='label+percent')])

    fig.update_layout(
        title=f'Sector Proportions on {date}',
        showlegend=True,
        legend=dict(title="Sectors")
    )

    return fig

# Call the function with the loaded data
plot_sector_proportions(portfolio_data, sectors)

In [ ]:
def plot_portfolio_growth(portfolio_data):
    dates = sorted(portfolio_data.keys())
    start_date = pd.to_datetime(dates[0])
    dates = [d for d in dates if pd.to_datetime(d) >= start_date]

    df_plot = pd.DataFrame({
        'Date': pd.to_datetime(dates),
        'Total Market Value': [float(portfolio_data[d]['summary']['total_market_value']) for d in dates],
        'Cash': [float(portfolio_data[d]['summary']['cash']) for d in dates],
        'Unrealized Gain': [float(portfolio_data[d]['summary']['unrealized_gain_loss']) for d in dates],
        'Daily Gain': [float(portfolio_data[d]['summary']['daily_gain']) for d in dates],
        'Cumulative Return (%)': [float(portfolio_data[d]['summary']['cumulative_return']) * 100 for d in dates],
        'ROI (%)': [float(portfolio_data[d]['summary']['roi']) * 100 for d in dates],
    })

    latest_date = df_plot['Date'].max()
    returns, available_periods = calculate_returns(df_plot, latest_date)

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    add_sp500_trace(fig, df_plot)
    add_portfolio_traces(fig, df_plot)
    add_performance_annotation(fig, returns, available_periods)

    # Apply layout updates
    fig.update_layout(
        title={'text': 'Portfolio Performance & Value vs S&P 500', 'y': 0.95, 'x': 0.5,
               'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Date',
        hovermode='x unified',
        showlegend=True,
        xaxis=dict(
            rangeslider=dict(visible=False),
            rangeselector=dict(
                buttons=get_range_selector_buttons(available_periods)
            )
        ),
        margin=dict(t=140),
    )
    fig.update_yaxes(title_text="Performance (%)",
                     ticksuffix='%', secondary_y=False)
    fig.update_yaxes(title_text="Value ($)", tickprefix='$', secondary_y=True)

    return fig


def calculate_returns(df_plot, latest_date):
    data_range = (latest_date - df_plot['Date'].min()).days
    available_periods = []
    returns = {}

    periods = [
        ('1D', 1), ('1W', 7), ('1M', 30), ('3M', 90),
        ('6M', 180), ('YTD', None), ('1Y', 365), ('2Y',
                                                  730), ('5Y', 1825), ('All', data_range)
    ]

    for period, days in periods:
        if period == 'YTD':
            ytd_start = pd.Timestamp(year=latest_date.year, month=1, day=1)
            if df_plot['Date'].min() <= ytd_start:
                returns[period] = calculate_return(
                    df_plot, latest_date, ytd=True)
                available_periods.append(period)
        elif data_range >= (days or 0):
            if period == 'All':
                returns[period] = df_plot['ROI (%)'].iloc[-1]
            else:
                returns[period] = calculate_return(
                    df_plot, latest_date, days=days)
            available_periods.append(period)

    return returns, available_periods


def calculate_return(df_plot, latest_date, days=None, ytd=False):
    if ytd:
        start_date = pd.Timestamp(year=latest_date.year, month=1, day=1)
    else:
        start_date = latest_date - pd.Timedelta(days=days)
    return df_plot['ROI (%)'].iloc[-1] - df_plot[df_plot['Date'] >= start_date]['ROI (%)'].iloc[0]


def add_sp500_trace(fig, df_plot):
    sp500_data = fetch_sp500_data(df_plot['Date'].min(), df_plot['Date'].max())
    sp500_performance = (
        (sp500_data['Close'] - sp500_data['Close'].iloc[0]) / sp500_data['Close'].iloc[0]) * 100
    fig.add_trace(
        go.Scatter(x=sp500_data.index, y=sp500_performance.round(2), line=dict(color='blue', width=2),
                   name='S&P 500', visible=True,
                   hovertemplate='S&P 500: %{y:.2f}%<extra></extra>'),
        secondary_y=False
    )


def add_portfolio_traces(fig, df_plot):
    fig.add_trace(
        go.Scatter(x=df_plot['Date'], y=df_plot['ROI (%)'].round(2), fill='tozeroy',
                   fillcolor='rgba(255,0,0,0.2)', line=dict(color='red', width=2), name='ROI',
                   hovertemplate='ROI: %{y:.2f}% (Cum. Return: %{customdata:.2f}%)<extra></extra>',
                   customdata=df_plot['Cumulative Return (%)'].round(2), visible=True),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(x=df_plot['Date'], y=df_plot['Total Market Value'].round(2),
                   line=dict(color='green', width=2), name='Market Value', visible=True,
                   hovertemplate='$%{y:,.2f} (Cash: $%{customdata:,.2f})',
                   customdata=df_plot['Cash'].round(2)),
        secondary_y=True
    )
    fig.add_trace(
        go.Bar(x=df_plot['Date'], y=df_plot['Unrealized Gain'].round(2), name='Unr. Gain',
               marker_color='gray', opacity=0.9,
               hovertemplate='$%{y:,.2f} (Daily: $%{customdata:,.2f})',
               customdata=df_plot['Daily Gain'].round(2)),
        secondary_y=True
    )


def add_performance_annotation(fig, returns, available_periods):
    performance_text = "Portfolio ROI:<br>"
    for i, period in enumerate(available_periods):
        if i % 2 == 0 and i != 0:
            performance_text += "<br>"
        performance_text += f"<span style='display: inline-block; width: 70px;'>{
            period}: {returns[period]:.2f}%</span>"
        if i % 2 == 0 and i != len(available_periods) - 1:
            performance_text += " | "

    fig.add_annotation(
        x=1, y=1.05, xref='paper', yref='paper', text=performance_text, showarrow=False,
        font=dict(size=10, family="monospace"), align='left', xanchor='right', yanchor='bottom',
        bgcolor='rgba(255,255,255,0.8)', bordercolor='rgba(0,0,0,0.5)', borderwidth=1, borderpad=4
    )


def get_range_selector_buttons(available_periods):
    button_map = {
        '1W': dict(count=7, label="1W", step="day", stepmode="backward"),
        '1M': dict(count=1, label="1M", step="month", stepmode="backward"),
        '3M': dict(count=3, label="3M", step="month", stepmode="backward"),
        '6M': dict(count=6, label="6M", step="month", stepmode="backward"),
        'YTD': dict(count=1, label="YTD", step="year", stepmode="todate"),
        '1Y': dict(count=1, label="1Y", step="year", stepmode="backward"),
        '2Y': dict(count=2, label="2Y", step="year", stepmode="backward"),
        '5Y': dict(count=5, label="5Y", step="year", stepmode="backward"),
        'All': dict(step="all", label="MAX")
    }
    return [button_map[period] for period in available_periods if period in button_map]


def fetch_sp500_data(start_date, end_date):
    end_date = pd.to_datetime(end_date) + timedelta(days=1)
    sp500 = yf.Ticker("^GSPC")
    sp500_data = sp500.history(start=start_date, end=end_date)
    return sp500_data[['Close']]

In [ ]:
plot_portfolio_growth(portfolio_data)

In [ ]:
def process_portfolio_data(portfolio_data, date=None, sort_by='Symbol', ascending=True):
    date = date or max(portfolio_data.keys())
    if date not in portfolio_data:
        raise ValueError(f"No data available for date: {date}")

    data = portfolio_data[date]
    holdings_data = process_holdings(data['holdings'])

    df = pd.DataFrame(holdings_data)
    df = df.sort_values(by=sort_by, ascending=ascending)

    return {
        'date': date,
        'holdings': df,
        'summary': data['summary'],
        'realized_gains': data['summary'].get('realized_gains', 0),
    }


def process_holdings(holdings):
    return [
        {
            'Symbol': symbol,
            'Quantity': round(float(info['quantity']), 2),
            'Unit Cost': round(float(info['unit_cost']), 2),
            'Market Price': round(float(info['market_price']), 2),
            'Total Cost': round(float(info['total_cost']), 2),
            'Market Value': round(float(info['market_value']), 2),
            'Unrealized G/L': f"${float(info['unrealized_gain_loss']):,.2f}",
            'Daily Gain': f"${float(info['daily_gain']):,.2f} ({float(info['daily_return'])*100:.2f}%)"
        }
        for symbol, info in holdings.items()
    ]

In [ ]:
def display_portfolio_analysis(processed_data):
    date = processed_data['date']
    df = processed_data['holdings']
    summary = processed_data['summary']

    # Display holdings
    display(HTML(f"<h2>Portfolio Status as of {date}</h2>"))
    display(HTML(df.to_html(index=False, classes='dataframe')))

    # Display summary
    print("\nPortfolio Summary:")
    print_summary_item("Total Portfolio Value",
                       summary.get('total_portfolio_value', 0))
    print_summary_item("Total Market Value",
                       summary.get('total_market_value', 0))
    print_summary_item("Cash", summary.get('cash', 0))
    print_summary_item("Total Deposit", summary.get('total_deposits', 0))
    print_summary_item("Unrealized Gain/Loss",
                       summary.get('unrealized_gain_loss', 0))
    print_summary_item("Daily Gain", summary.get('daily_gain', 0),
                       percentage=summary.get('daily_return', 0))
    print_summary_item("Total Return", summary.get(
        'total_return', 0), is_percentage=True)
    print_summary_item("Cumulative Return", summary.get(
        'cumulative_return', 0), is_percentage=True)
    print_summary_item("ROI", summary.get('roi', 0), is_percentage=True)


def print_summary_item(label, value, percentage=None, is_percentage=False):
    if is_percentage:
        print(f"{label}: {float(value)*100:.2f}%")
    elif percentage is not None:
        print(f"{label}: ${float(value):,.2f} ({float(percentage)*100:.2f}%)")
    else:
        print(f"{label}: ${float(value):,.2f}")

In [ ]:
processed_data = process_portfolio_data(portfolio_data)
display_portfolio_analysis(processed_data)

In [ ]:
# processed_data = process_portfolio_data(portfolio_data, date="2024-04-10")
# display_portfolio_analysis(processed_data)

In [ ]:
def process_transaction_history(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    holdings = defaultdict(
        lambda: {"quantity": Decimal('0'), "cost": Decimal('0')})
    closed_positions = {}
    partially_sold_positions = defaultdict(dict)

    for date, transactions in sorted(data.items()):
        # Process sells
        for symbol, sells in transactions.get("sell", {}).items():
            for sell in sells:
                quantity_sold = Decimal(str(sell['quantity']))
                sell_amount = Decimal(str(sell['amount']))

                if holdings[symbol]["quantity"] >= quantity_sold:
                    cost_per_unit = holdings[symbol]["cost"] / \
                        holdings[symbol]["quantity"]
                    cost_of_sold_shares = cost_per_unit * quantity_sold
                    profit_loss = sell_amount - cost_of_sold_shares

                    holdings[symbol]["quantity"] -= quantity_sold
                    holdings[symbol]["cost"] -= cost_of_sold_shares

                    if holdings[symbol]["quantity"] < Decimal('0.00001'):
                        if symbol in partially_sold_positions:
                            closed_positions[symbol] = {
                                "close_date": date,
                                "quantity": float(partially_sold_positions[symbol]["quantity"] + quantity_sold),
                                "cost": float(partially_sold_positions[symbol]["cost"] + cost_of_sold_shares),
                                "profit_loss": float(partially_sold_positions[symbol]["profit_loss"] + profit_loss)
                            }
                            del partially_sold_positions[symbol]
                        else:
                            closed_positions[symbol] = {
                                "close_date": date,
                                "quantity": float(quantity_sold),
                                "cost": float(cost_of_sold_shares),
                                "profit_loss": float(profit_loss)
                            }
                        del holdings[symbol]
                    else:
                        if symbol in partially_sold_positions:
                            partially_sold_positions[symbol]["quantity"] += quantity_sold
                            partially_sold_positions[symbol]["cost"] += cost_of_sold_shares
                            partially_sold_positions[symbol]["profit_loss"] += profit_loss
                            partially_sold_positions[symbol]["last_sell_date"] = date
                        else:
                            partially_sold_positions[symbol] = {
                                "last_sell_date": date,
                                "quantity": quantity_sold,
                                "cost": cost_of_sold_shares,
                                "profit_loss": profit_loss
                            }

        # Process buys
        for symbol, buys in transactions.get("buy", {}).items():
            for buy in buys:
                quantity_bought = Decimal(str(buy['quantity']))
                amount_spent = Decimal(str(buy['amount']))
                holdings[symbol]["quantity"] += quantity_bought
                holdings[symbol]["cost"] += amount_spent

        # Process reinvestments
        for symbol, reinvestments in transactions.get("reinvestment", {}).items():
            for reinvestment in reinvestments:
                quantity_reinvested = Decimal(str(reinvestment['quantity']))
                amount_reinvested = Decimal(str(reinvestment['amount']))
                holdings[symbol]["quantity"] += quantity_reinvested
                holdings[symbol]["cost"] += amount_reinvested

        # Process splits
        for symbol, split_info in transactions.get("split", {}).items():
            if symbol in holdings:
                new_quantity = Decimal(str(split_info['quantity']))
                old_quantity = holdings[symbol]["quantity"]
                split_ratio = new_quantity / old_quantity
                holdings[symbol]["quantity"] = new_quantity
                holdings[symbol]["cost"] *= split_ratio

    # Convert Decimal values to float for JSON serialization
    for symbol in partially_sold_positions:
        partially_sold_positions[symbol] = {
            k: float(v) if isinstance(v, Decimal) else v
            for k, v in partially_sold_positions[symbol].items()
        }

    total_realized_gain_loss = sum(position['profit_loss'] for position in closed_positions.values()) + \
        sum(position['profit_loss']
            for position in partially_sold_positions.values())
    total_cost_basis = sum(position['cost'] for position in closed_positions.values()) + \
        sum(position['cost'] for position in partially_sold_positions.values())

    # Calculate percentage gain/loss
    percentage_gain_loss = (total_realized_gain_loss /
                            total_cost_basis) * 100 if total_cost_basis != 0 else 0

    return dict(holdings), [
        {
            "symbol": symbol,
            "close_date": info["close_date"],
            "quantity": info["quantity"],
            "cost": info["cost"],
            "profit_loss": info["profit_loss"]
        }
        for symbol, info in closed_positions.items()
    ], partially_sold_positions, total_realized_gain_loss, percentage_gain_loss


def create_dataframes(positions, closed_positions, partially_sold_positions, total_realized_gain_loss, percentage_gain_loss):
    closed_positions_df = pd.DataFrame(closed_positions)

    partially_sold_positions_df = pd.DataFrame([
        {
            'symbol': symbol,
            'last_sell_date': data['last_sell_date'],
            'quantity': data['quantity'],
            'cost': data['cost'],
            'profit_loss': data['profit_loss']
        }
        for symbol, data in partially_sold_positions.items()
    ])

    def format_profit_loss(row):
        profit_loss = row['profit_loss']
        cost = row['cost']
        percentage = (profit_loss / cost) * 100
        return f"${profit_loss:.2f} ({percentage:.2f}%)"

    closed_positions_df['profit_loss'] = closed_positions_df.apply(
        format_profit_loss, axis=1)
    partially_sold_positions_df['profit_loss'] = partially_sold_positions_df.apply(
        format_profit_loss, axis=1)

    closed_positions_df = closed_positions_df.rename(
        columns={
            'symbol': 'Symbol',
            'close_date': 'Close Date',
            'quantity': 'Quantity',
            'cost': 'Cost',
            'profit_loss': 'Profit/Loss'
        }
    )

    active_positions_df = partially_sold_positions_df.rename(columns={
        'symbol': 'Symbol',
        'last_sell_date': 'Last Sell Date',
        'quantity': 'Quantity Sold',
        'cost': 'Cost',
        'profit_loss': 'Profit/Loss'
    })

    # Format the monetary values
    for df in [closed_positions_df, active_positions_df]:
        df['Cost'] = df['Cost'].apply(lambda x: f"${x:.2f}")

    # Remove the index
    closed_positions_df = closed_positions_df.reset_index(drop=True)
    active_positions_df = active_positions_df.reset_index(drop=True)

    return closed_positions_df, active_positions_df, total_realized_gain_loss, percentage_gain_loss


# Usage
positions, closed_positions, partially_sold_positions, total_realized_gain_loss, percentage_gain_loss = process_transaction_history(
    'transaction_history.json')
closed_positions_df, active_positions_df, total_realized_gain_loss, percentage_gain_loss = create_dataframes(
    positions, closed_positions, partially_sold_positions, total_realized_gain_loss, percentage_gain_loss)

print("Closed Positions:")
display(closed_positions_df)

print("\nPartially Sold Positions:")
display(active_positions_df)

print(f"\nTotal Realized Gains/Losses: ${
      total_realized_gain_loss:.2f} ({percentage_gain_loss:.2f}%)")

In [ ]:
def plot_asset_allocation_over_time(portfolio_data, sort_by='percentage'):
    dates = sorted(portfolio_data.keys())
    active_stocks = get_active_stocks(portfolio_data[dates[-1]]['holdings'])
    data = calculate_asset_proportions(portfolio_data, dates, active_stocks)

    fig = go.Figure()
    add_asset_traces(fig, dates, data, portfolio_data, sort_by)
    update_layout(fig, sort_by)

    return fig


def get_active_stocks(latest_holdings):
    return [stock for stock, info in latest_holdings.items()
            if float(info['market_value']) > 0]


def calculate_asset_proportions(portfolio_data, dates, active_stocks):
    data = {stock: [] for stock in active_stocks}
    data['Cash'] = []

    for date in dates:
        holdings = portfolio_data[date]['holdings']
        valid_holdings = {stock: info for stock, info in holdings.items()
                          if not math.isnan(float(info['market_value']))}

        cash = float(portfolio_data[date]['summary']['cash'])
        total_value = sum(float(info['market_value'])
                          for info in valid_holdings.values()) + cash

        for stock in active_stocks:
            if stock in valid_holdings:
                value = float(valid_holdings[stock]['market_value'])
                proportion = (value / total_value) * \
                    100 if total_value > 0 else 0
            else:
                proportion = 0
            data[stock].append(proportion)

        cash_proportion = (cash / total_value) * 100 if total_value > 0 else 0
        data['Cash'].append(cash_proportion)

    return data


def add_asset_traces(fig, dates, data, portfolio_data, sort_by):
    if sort_by == 'date':
        sorted_stocks = sort_by_add_date(portfolio_data, data.keys())
    else:  # sort_by == 'percentage'
        sorted_stocks = sort_by_percentage(data)

    # Remove Cash from sorted_stocks if it's there
    sorted_stocks = [stock for stock in sorted_stocks if stock != 'Cash']

    # Add traces in reverse order, with Cash always at the end (top of the stack)
    for stock in reversed(sorted_stocks):
        add_trace(fig, dates, data, stock)

    # Always add Cash last (it will appear at the top)
    add_trace(fig, dates, data, 'Cash', '#FFD700')


def sort_by_add_date(portfolio_data, stocks):
    first_appearance = {}
    for date in sorted(portfolio_data.keys()):
        for stock in stocks:
            if stock not in first_appearance and stock != 'Cash':
                if stock in portfolio_data[date]['holdings']:
                    first_appearance[stock] = date
    return sorted(first_appearance, key=first_appearance.get)


def sort_by_percentage(data):
    latest_percentages = {stock: values[-1]
                          for stock, values in data.items() if stock != 'Cash'}
    return sorted(latest_percentages, key=latest_percentages.get, reverse=True)


def add_trace(fig, dates, data, stock, color=None):
    proportions = data[stock]
    hover_text = [f"{stock}: {proportion:.2f}%" if not math.isnan(
        proportion) else f"{stock}: N/A" for proportion in proportions]

    fig.add_trace(go.Scatter(
        x=dates,
        y=proportions,
        mode='lines',
        stackgroup='one',
        name=stock,
        hoverinfo='text',
        text=hover_text,
        line=dict(color=color) if color else dict(),
    ))


def update_layout(fig, sort_by):
    title = 'Asset Allocation Over Time (Sorted by {})'.format(
        'Latest Percentage' if sort_by == 'percentage' else 'Add Date')
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis=dict(
            title='Proportion',
            tickformat='.0%',
            range=[0, 100],
            tickvals=[0, 20, 40, 60, 80, 100],
            ticktext=['0%', '20%', '40%', '60%', '80%', '100%']
        ),
        legend_title='Assets',
        hovermode='x unified'
    )

In [ ]:
# Sort by percentage (default)
# plot_asset_allocation_over_time(portfolio_data)

# Or sort by add date
# plot_asset_allocation_over_time(portfolio_data, sort_by='date')

plot_asset_allocation_over_time(portfolio_data)

In [ ]:
def generate_html_report():
    dates = sorted(portfolio_data.keys())
    latest_date = dates[-1]

    stock_proportions_plot = plot_stock_proportions(
        portfolio_data, latest_date)
    sector_proportions_plot = plot_sector_proportions(
        portfolio_data, sectors, latest_date)
    portfolio_growth_plot = plot_portfolio_growth(portfolio_data)
    asset_allocation_over_time_plot = plot_asset_allocation_over_time(
        portfolio_data)

    # Apply monospace font to all plots
    for plot in [stock_proportions_plot, sector_proportions_plot, portfolio_growth_plot, asset_allocation_over_time_plot]:
        plot.update_layout(font=dict(family="monospace", weight='bold'))

    processed_data = process_portfolio_data(portfolio_data, latest_date)

    positions, closed_positions, partially_sold_positions, total_realized_gain_loss, percentage_gain_loss = process_transaction_history(
        'transaction_history.json')
    closed_positions_df, active_positions_df, total_realized_gain_loss, percentage_gain_loss = create_dataframes(
        positions, closed_positions, partially_sold_positions, total_realized_gain_loss, percentage_gain_loss)

    # Convert DataFrames to HTML strings
    closed_positions_table = closed_positions_df.to_html(
        index=False, classes='dataframe')
    partially_sold_positions_table = active_positions_df.to_html(
        index=False, classes='dataframe')
    portfolio_table = processed_data['holdings'].to_html(
        index=False, classes='dataframe')

    # Format portfolio summary
    summary = processed_data['summary']
    format_dict = {
        'date': latest_date,
        'portfolio_table': portfolio_table,
        'portfolio_growth_plot': to_html(portfolio_growth_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'stock_proportions_plot': to_html(stock_proportions_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'sector_proportions_plot': to_html(sector_proportions_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'asset_allocation_over_time_plot': to_html(asset_allocation_over_time_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'date_options': ' '.join(f'<option value="{date}">{date}</option>' for date in dates),
        'cash': f"${float(summary.get('cash', 0)):,.2f}",
        'total_portfolio_value': f"${float(summary.get('total_portfolio_value', 0)):,.2f}",
        'total_market_value': f"${float(summary.get('total_market_value', 0)):,.2f}",
        'unrealized_gain_loss': f"${float(summary.get('unrealized_gain_loss', 0)):,.2f}",
        'daily_gain': f"${float(summary.get('daily_gain', 0)):,.2f} ({float(summary.get('daily_return', 0))*100:.2f}%)",
        'total_return': f"{float(summary.get('total_return', 0))*100:.2f}%",
        'cumulative_return': f"{float(summary.get('cumulative_return', 0))*100:.2f}%",
        'roi': f"{float(summary.get('roi', 0))*100:.2f}%",
        'total_deposits': f"${float(summary.get('total_deposits', 0)):,.2f}",
        'closed_positions_table': closed_positions_table,
        'partially_sold_positions_table': partially_sold_positions_table,
        'total_realized_gain_loss': f"${total_realized_gain_loss:.2f} ({percentage_gain_loss:.2f}%)",
    }
    # Read the HTML template
    template_path = 'reports/templates/portfolio_report_template.html'
    with open(template_path, 'r') as template_file:
        html_template = template_file.read()

    # Read the CSS content
    css_path = 'reports/templates/portfolio_report_styles.css'
    with open(css_path, 'r') as css_file:
        css_content = css_file.read()

    # Escape curly braces in the CSS content
    css_content_escaped = css_content.replace('{', '{{').replace('}', '}}')

    # Replace the CSS placeholder with the escaped CSS content
    html_template = html_template.replace(
        '/* CSS_PLACEHOLDER */', css_content_escaped)

    # Remove whitespace from placeholders
    html_template = re.sub(r'\{\s*(\w+)\s*\}', r'{\1}', html_template)

    # Generate the HTML content
    html_content = html_template.format(**format_dict)

    # Ensure the 'reports' directory exists
    os.makedirs('reports', exist_ok=True)

    # Generate only the 'latest' version
    today_date = datetime.now().strftime("%Y-%m-%d")
    filename = f'portfolio_report_latest_{today_date}.html'
    with open(os.path.join('reports', filename), 'w') as f:
        f.write(html_content)

    print(f"HTML report generated: reports/{filename}")


generate_html_report()